# Градиентный бустинг "в лоб"¶

### 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [47]:
import json
import bz2
import pandas
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [18]:
features = pandas.read_csv('./data/features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [19]:
features = features.drop(['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)
y = features['radiant_win']
features = features.drop(['radiant_win'], axis=1)
features.tail()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,0,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,2,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0
114405,1450292986,1,100,3,1135,766,6,0,2,6,...,0,-21.0,254.0,-85.0,183.0,5,3,3,1,-42.0
114406,1450313370,7,50,3,1053,799,7,0,0,7,...,0,-36.0,131.0,-59.0,NaN,5,3,2,0,-39.0


### 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. 
Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [20]:
s = features.count()
s[s != 97230]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

* `first_blood_time` - первое убийство произошло не на первых 5 минутах.
* `dire_courier_time` - за пять минут саппорт не купил курьера (потратил начальные деньги на что то другое).

In [21]:
s = features.count(numeric_only=True)
s[s != 97230]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

### 3. Замените пропуски на нули с помощью функции fillna(). 
На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [22]:
features_negative = features.fillna(-100_000)
features_zero = features.fillna(0)
s = features_negative.count()
s[s != 97230]

Series([], dtype: int64)

### 4. Какой столбец содержит целевую переменную? Запишите его название.
**radiant_win**

### 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". 
Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [23]:
clf = GradientBoostingClassifier(random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)
grid = {
    'n_estimators': np.arange(10, 110, 10),
    'learning_rate': [0.5, 0.3, 0.2, 0.1]
}
gs1 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [24]:
start_time1 = datetime.datetime.now()
gs1.fit(features_negative, y)
end_time1 = datetime.datetime.now()

/home/worker/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [25]:
gs1.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.3, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=241,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [26]:
end_time1 - start_time1

datetime.timedelta(seconds=2417, microseconds=348311)

In [27]:
for k, v in gs1.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [ 9.30503783 17.49604387 25.89590874 34.01634316 41.85479088 50.41099668
 58.51432915 66.47973175 74.13282652 82.7943809   9.17609181 17.59402361
 25.95510731 34.27470698 42.45581002 50.82619429 58.64613299 67.23825865
 75.21503773 83.83333325  9.05469375 17.52939715 25.8152657  34.38746223
 42.84111581 50.8216414  59.12212663 67.73066087 75.87272663 84.06612458
  9.10504785 17.50226355 25.78174725 33.80906081 42.33916025 50.38270679
 59.02498035 67.54668632 75.8692718  75.77647724]

Key: mean_score_time, value: [0.04979239 0.05284591 0.06136379 0.07763524 0.08759227 0.09471784
 0.11012154 0.12200222 0.13017917 0.13840399 0.04681511 0.05925918
 0.07119803 0.08192463 0.09153662 0.10274901 0.11159921 0.12419367
 0.1341784  0.14466133 0.04909868 0.05971527 0.07109017 0.08092217
 0.09355693 0.10364404 0.11372967 0.1255435  0.13419495 0.14568081
 0.04937596 0.06064086 0.07100425 0.08105454 0.09328604 0.1043107
 0.1133337  0.12458034 0.13459477 0.11934247]

Key: me

In [28]:
clf = GradientBoostingClassifier(random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)
grid = {
    'n_estimators': np.arange(10, 110, 10),
    'learning_rate': [0.5, 0.3, 0.2, 0.1]
}
gs2 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [29]:
start_time2 = datetime.datetime.now()
gs2.fit(features_zero, y)
end_time2 = datetime.datetime.now()

In [30]:
gs2.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.3, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=241,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [31]:
end_time2 - start_time2

datetime.timedelta(seconds=2403, microseconds=350657)

In [32]:
for k, v in gs2.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [ 9.27182493 17.55245519 25.74419861 33.82147002 42.10020485 50.26153522
 58.01008654 66.00227985 74.14035668 82.10844016  9.05429235 17.4098546
 25.7934617  34.22803025 42.25757833 50.38922243 58.30991044 66.93005867
 74.9811007  83.07439623  9.11068454 17.56110392 25.83403397 34.26803932
 42.54527183 51.0147285  58.98514171 67.17904477 75.42689786 83.73749762
  9.11657414 17.50992908 25.78885965 33.86154709 42.24624057 50.75771284
 58.96899738 67.54896264 75.8113318  77.10922008]

Key: mean_score_time, value: [0.05002489 0.06194267 0.07238097 0.08063989 0.09364715 0.1020916
 0.11356983 0.12216282 0.12852621 0.14477301 0.05164533 0.0604507
 0.07260385 0.08463988 0.0924859  0.10392857 0.11148901 0.12363172
 0.1341022  0.14593434 0.04976768 0.06041369 0.07018709 0.08007612
 0.09354286 0.10344324 0.11452374 0.12594604 0.13653283 0.14672508
 0.04953418 0.06033015 0.07028923 0.0821569  0.09124579 0.10181599
 0.1127913  0.1240159  0.13593659 0.11983948]

Key: mean

### Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти выше по тексту. Какое качество при этом получилось?

In [37]:
gs3 = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                                 learning_rate=0.3, loss='deviance', max_depth=3,
                                 max_features=None, max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None,
                                 min_samples_leaf=1, min_samples_split=2,    
                                 min_weight_fraction_leaf=0.0, n_estimators=30,
                                 n_iter_no_change=None, presort='auto', random_state=241,
                                 subsample=1.0, tol=0.0001, validation_fraction=0.1,
                                 verbose=0, warm_start=False)
X_train, X_test, y_train, y_test = train_test_split(features_zero, y, 
                                                    test_size=0.8, random_state=241)

In [49]:
%%time
gs3.fit(X_train, y_train)

CPU times: user 3.32 s, sys: 7.97 ms, total: 3.33 s
Wall time: 3.33 s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.3, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              n_iter_no_change=None, presort='auto', random_state=241,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [50]:
roc_auc_score(y_test, gs3.predict_proba(X_test)[:, 1])

0.6955173766656932